In [1]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import LabelEncoder

# from pytorchtools_st import EarlyStopping

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd drive/MyDrive/datasets

/content/drive/MyDrive/datasets


## 1. Генерирование русских имен при помощи RNN

Датасет: https://disk.yandex.ru/i/2yt18jHUgVEoIw

1.1 На основе файла name_rus.txt создайте датасет.
  * Учтите, что имена могут иметь различную длину
  * Добавьте 4 специальных токена: 
    * `<PAD>` для дополнения последовательности до нужной длины;
    * `<UNK>` для корректной обработки ранее не встречавшихся токенов;
    * `<SOS>` для обозначения начала последовательности;
    * `<EOS>` для обозначения конца последовательности.
  * Преобразовывайте строку в последовательность индексов с учетом следующих замечаний:
    * в начало последовательности добавьте токен `<SOS>`;
    * в конец последовательности добавьте токен `<EOS>` и, при необходимости, несколько токенов `<PAD>`;
  * `Dataset.__get_item__` возращает две последовательности: последовательность для обучения и правильный ответ. 
  
  Пример:
  ```
  s = 'The cat sat on the mat'
  # преобразуем в индексы
  s_idx = [2, 5, 1, 2, 8, 4, 7, 3, 0, 0]
  # получаем x и y (__getitem__)
  x = [2, 5, 1, 2, 8, 4, 7, 3, 0]
  y = [5, 1, 2, 8, 4, 7, 3, 0, 0]
  ```

1.2 Создайте и обучите модель для генерации фамилии.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`;
  * Используйте рекуррентные слои;
  * Задача ставится как предсказание следующего токена в каждом примере из пакета для каждого момента времени. Т.е. в данный момент времени по текущей подстроке предсказывает следующий символ для данной строки (задача классификации);
  * Примерная схема реализации метода `forward`:
  ```
    input_X: [batch_size x seq_len] -> nn.Embedding -> emb_X: [batch_size x seq_len x embedding_size]
    emb_X: [batch_size x seq_len x embedding_size] -> nn.RNN -> output: [batch_size x seq_len x hidden_size] 
    output: [batch_size x seq_len x hidden_size] -> torch.Tensor.reshape -> output: [batch_size * seq_len x hidden_size]
    output: [batch_size * seq_len x hidden_size] -> nn.Linear -> output: [batch_size * seq_len x vocab_size]
  ```

1.3 Напишите функцию, которая генерирует фамилию при помощи обученной модели:
  * Построение начинается с последовательности единичной длины, состоящей из индекса токена `<SOS>`;
  * Начальное скрытое состояние RNN `h_t = None`;
  * В результате прогона последнего токена из построенной последовательности через модель получаете новое скрытое состояние `h_t` и распределение над всеми токенами из словаря;
  * Выбираете 1 токен пропорционально вероятности и добавляете его в последовательность (можно воспользоваться `torch.multinomial`);
  * Повторяете эти действия до тех пор, пока не сгенерирован токен `<EOS>` или не превышена максимальная длина последовательности.

При обучении каждые `k` эпох генерируйте несколько фамилий и выводите их на экран.

In [ ]:
weights = torch.tensor([0, 10, 40, 1], dtype=torch.float) # create a tensor of weights
torch.multinomial(weights, 4)

tensor([2, 1, 3, 0])

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/name_rus.txt', encoding = 'cp1251', header = None, names = ['Name_rus'])
display(df.head())
df.shape

,Name_rus
0,авдокея
1,авдоким
2,авдоня
3,авдотька
4,авдотьюшка


(1988, 1)

In [ ]:
class Vocab:
  def __init__(self, data):
    tokens = set()
    for name in data:
      tokens.update(name)

    self.max_seq_len = max([len(name) for name in data]) + 2 # Т.к. будут добавлены токены <SOS> and <EOS>

    self.idx_to_token = {(idx + 4): token for idx, token in enumerate(tokens)}
    self.idx_to_token[0] = '<PAD>' # для дополнения последовательности до нужной длины
    self.idx_to_token[1] = '<UNK>' # для корректной обработки ранее не встречавшихся токенов
    self.idx_to_token[2] = '<SOS>' # для обозначения начала последовательности
    self.idx_to_token[3] = '<EOS>' # для обозначения конца последовательности
    self.token_to_idx = {token: idx for idx, token in self.idx_to_token.items()}
    self.vocab_len = len(self.idx_to_token)

In [ ]:
class NameRusDataset(Dataset):
  def __init__(self, X, vocab: Vocab):
    self.X = X
    self.vocab = vocab

  def vectorize(self, name):
    name_t = torch.zeros(self.vocab.max_seq_len, dtype = int)
    
    name_t[0] =  self.vocab.token_to_idx['<SOS>']
    for idx, char in enumerate(name, 1): # начинаем с единицы, т.к. токен 0-ая позиция занята <SOS> 
      if char in self.vocab.token_to_idx:
        name_t[idx] = self.vocab.token_to_idx[char]
      else:
        name_t[idx] = self.vocab.token_to_idx['<UNK>']
        
    name_t[idx + 1] = self.vocab.token_to_idx['<EOS>']

    return name_t

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    vector = self.vectorize(self.X[idx])
    return vector[:-1], vector[1:]

In [ ]:
vocab = Vocab(df['Name_rus'])
vocab.max_seq_len, vocab.vocab_len

(15, 34)

In [ ]:
dataset = NameRusDataset(list(df['Name_rus']), vocab)
dataset[0]

(tensor([ 2,  6,  8, 13, 15, 25,  4, 10,  3,  0,  0,  0,  0,  0]),
 tensor([ 6,  8, 13, 15, 25,  4, 10,  3,  0,  0,  0,  0,  0,  0]))

In [ ]:
class CharRNN(nn.Module):
  def __init__(self, vocab_size: int, embedding_dim: int, hidden_size: int):
    super().__init__()

    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.hidden_size = hidden_size

    self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
    self.rnn = nn.GRU(input_size = embedding_dim, 
                      hidden_size = hidden_size,
                      batch_first = True) 
    self.fc = nn.Linear(hidden_size, vocab_size)
    self.dropout = nn.Dropout(p=0.25)

  def forward(self, input_tensor):
    # input_tensor: batch_size x seq_len
    embedded = self.embeddings(input_tensor)
    batch_size, seq_len, embedding_size = embedded.shape
    # embedded: batch_size x seq_len x embedding_size
    output, hidden = self.rnn(embedded)
    # output: batch_size x seq_len x hidden_size
    # hidden: 1 x batch_size x hidden_size
    output = output.reshape(batch_size*seq_len, self.hidden_size)
    output = self.dropout(output)
    # output: batch_size*seq_len x hidden_size
    output = self.fc(output)
    # output: batch_size*seq_len x vocab_size

    return output

In [ ]:
dataset_train = NameRusDataset(list(df['Name_rus']), vocab)
dataloader = DataLoader(dataset_train, batch_size = 32, shuffle = True, drop_last = True)

In [ ]:
def generate(model, vocab: Vocab):
  indices = [2]
  h_t = None
  sample_size = vocab.max_seq_len

  model.eval()
  for step in range(sample_size):
    x_t = torch.LongTensor([indices[step]]).unsqueeze(dim=0).to(device=device)

    # print(x_t.shape)
    x_emp = model.embeddings(x_t)
    # print(x_emp.shape)
    out_t, h_t = model.rnn(x_emp, h_t)
    out_t = out_t.squeeze(dim=1)
    # print(out_t.shape)
    predictions = model.fc(out_t).softmax(dim=1)
    # print(predictions.shape)
    char_idx = torch.multinomial(predictions, num_samples=1)
    # print(char_idx.shape)
    if char_idx == 3:
      break
    indices.append(char_idx.item())
  return ''.join(vocab.idx_to_token[token_idx] for token_idx in indices if token_idx != 2).capitalize()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CharRNN(vocab_size = vocab.vocab_len,
                hidden_size=32,
                embedding_dim=32).to(device)
criteriation = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

n_epochs = 20
for epoch in range(n_epochs):
  epoch_loss = 0
  model.train()
  for X_batch, y_barch in dataloader:
    X_batch = X_batch.to(device=device)
    y_barch = y_barch.to(device=device) # batch_size x seq_len
    predictions = model(X_batch)
    # predictions: batch_size*seq_len x vocab_size
    y_barch = y_barch.reshape(-1)
    # y_batch: batch_size*seq_len
    loss = criteriation(predictions, y_barch)
    loss.backward()
    epoch_loss += loss.item()

    optimizer.step()
    optimizer.zero_grad()
  model.eval()
  print(f"Epoch #{epoch} loss = {epoch_loss / len(dataloader)} Example: {generate(model)}")

Epoch #0 loss = 3.4550187472374208 Example: <sos>м<sos>оз<unk>нтхбууюухыэртгю
Example:
Epoch #1 loss = 3.381063195966905 Example: <sos>впхэжабырен<pad>вйьаь<pad>йа
Example:
Epoch #2 loss = 3.304799537504873 Example: <sos>агцдфьяечрюнвгьодбаю
Example:
Epoch #3 loss = 3.2162345186356576 Example: <sos>
Example:
Epoch #4 loss = 3.116611865258986 Example: <sos>гйутзьююцпт<pad>уттгчяоб
Example:
Epoch #5 loss = 3.0133990741545156 Example: <sos>зеб<unk>кффкьена
Example:
Epoch #6 loss = 2.9213622500819545 Example: <sos>епчзлбеюлшжьго
Example:
Epoch #7 loss = 2.8509089716019167 Example: <sos>о
Example:
Epoch #8 loss = 2.798775311439268 Example: <sos>д
Example:
Epoch #9 loss = 2.7576251106877483 Example: <sos>еоа
Example:
Epoch #10 loss = 2.7212421317254343 Example: <sos>снозя
Example:
Epoch #11 loss = 2.689467168623401 Example: <sos>фифснолнси
Example:
Epoch #12 loss = 2.6585454979250507 Example: <sos>врлюя
Example:
Epoch #13 loss = 2.630206650303256 Example: <sos>антакф
Example:
Epoch #14 loss 

## 2. Генерирование текста при помощи RNN

2.1 Скачайте из интернета какое-нибудь художественное произведение
  * Выбирайте достаточно крупное произведение, чтобы модель лучше обучалась;

2.2 На основе выбранного произведения создайте датасет. 

Отличия от задачи 1:
  * Токены <SOS>, `<EOS>` и `<UNK>` можно не добавлять;
  * При создании датасета текст необходимо предварительно разбить на части. Выберите желаемую длину последовательности `seq_len` и разбейте текст на построки длины `seq_len` (можно без перекрытия, можно с небольшим перекрытием).

2.3 Создайте и обучите модель для генерации текста
  * Задача ставится точно так же как в 1.2;
  * При необходимости можете применить:
    * двухуровневые рекуррентные слои (`num_layers`=2)
    * [обрезку градиентов](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)

2.4 Напишите функцию, которая генерирует фрагмент текста при помощи обученной модели
  * Процесс генерации начинается с небольшого фрагмента текста `prime`, выбранного вами (1-2 слова) 
  * Сначала вы пропускаете через модель токены из `prime` и генерируете на их основе скрытое состояние рекуррентного слоя `h_t`;
  * После этого вы генерируете строку нужной длины аналогично 1.3


In [ ]:
df = pd.DataFrame(columns = ['Part_text'])
txt = ''
with open('/content/drive/MyDrive/datasets/AnnaKarenina__.txt', 'r') as f:
  for line in f:
    l = re.sub(r"\n", " ", line)
    txt += ' ' + ' '.join(l.split())

txt = txt.split()

k = 0
for idx in range(0, len(txt), 100):
  if idx + 100 <= len(txt):
    df.loc[k, 'Part_text'] = ' '.join(txt[idx: idx + 100])
  else:
    df.loc[k, 'Part_text'] = ' '.join(txt[idx:])
  k += 1

display(df)
df.shape

,Part_text
0,Лев Толстой Анна Каренина Роман «Широкого дыха...
1,находил в новом романе Толстого «огромную псих...
2,в объявлении не называть его сочинение романом...
3,"начиная работу, он говорил Н.Н. Страхову: «…ро..."
4,"70-е же годы, в эпоху глубокого социального кр..."
...,...
2947,"48). 322 …читая Шопенгауера, он подставил на м..."
2948,катехизисного изложения учения о церкви» и «Мы...
2949,и в одной из последних повестей – «После бала»...
2950,332 Т.А. Кузминская. Моя жизнь дома и в Ясной ...


(2952, 1)

In [ ]:
df.to_csv('/content/drive/MyDrive/datasets/AnnaKarenina_conclusion.csv')

In [44]:
df = pd.read_csv('/content/drive/MyDrive/datasets/AnnaKarenina_conclusion.csv')
df

,Unnamed: 0,Part_text
0,0,Лев Толстой Анна Каренина Роман «Широкого дыха...
1,1,находил в новом романе Толстого «огромную псих...
2,2,в объявлении не называть его сочинение романом...
3,3,"начиная работу, он говорил Н.Н. Страхову: «…ро..."
4,4,"70-е же годы, в эпоху глубокого социального кр..."
...,...,...
2947,2947,"48). 322 …читая Шопенгауера, он подставил на м..."
2948,2948,катехизисного изложения учения о церкви» и «Мы...
2949,2949,и в одной из последних повестей – «После бала»...
2950,2950,332 Т.А. Кузминская. Моя жизнь дома и в Ясной ...


In [45]:
class Vocab:
  def __init__(self, data):
    tokens = [] # Все слова в словаре
    for txt in data:
      # for sent in sent_tokenize(txt): # лексемазация предложения
      for word in word_tokenize(txt): # лексемазация слов
        tokens.append(word)

    tokens = set(tokens)

    self.max_seq_len = max([len(word_tokenize(txt)) for txt in data]) + 1

    self.idx_to_token = {(idx + 2): token for idx, token in enumerate(tokens)}
    self.idx_to_token[0] = '<PAD>' # для дополнения последовательности до нужной длины
    self.idx_to_token[1] = '<SOS>' # для обозначения начала последовательности
    self.token_to_idx = {token: idx for idx, token in self.idx_to_token.items()} # токен слово
    self.vocab_len = len(self.idx_to_token)

In [46]:
vocab = Vocab(df['Part_text'])
vocab.vocab_len, vocab.max_seq_len

(39103, 184)

In [47]:
class NameRusDataset(Dataset):
  def __init__(self, X: list, vocab: Vocab):
    self.X = X
    self.vocab = vocab

  def vectorize(self, seq):
    seq_t = torch.zeros(self.vocab.max_seq_len, dtype = int)
    
    seq_t[0] =  self.vocab.token_to_idx['<SOS>']
    for idx, char in enumerate(word_tokenize(seq), 1): # начинаем с единицы, т.к. токен 0-ая позиция занята <SOS> 
      seq_t[idx] = self.vocab.token_to_idx[char]

    return seq_t

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    vector = self.vectorize(self.X[idx])
    return vector[:-1], vector[1:]

In [48]:
dataset = NameRusDataset(list(df['Part_text']), vocab)
dataset[0]

(tensor([    1, 23784,  5271, 12473, 24554, 23959,  6124,    66, 39030, 24274,
          6124, 12473, 24554, 24274, 35254, 31099,  6124, 28301, 27794, 24274,
         26068, 14612, 22515, 31423, 24056, 11573, 10447, 29784, 12822, 26134,
         36064, 38938, 26086, 31746, 12652, 11743, 28254, 24797, 26068,  4201,
         35758, 12776,  4335, 30067, 16198,  3569, 16198, 18589, 27889, 25473,
          6124, 17309, 12822, 27042, 31423, 18986, 13186, 11466, 36919, 31423,
           913, 12822, 27042, 31423, 18986, 16535, 19794, 24797, 12822, 23492,
         31423, 25318, 17309, 11132, 16863, 17687, 30222, 24274, 27510, 19847,
         31423, 35819, 24047, 26068, 15354, 13764, 21615, 15729, 18675, 10103,
         18878, 17438, 28617, 27510, 12998, 31423, 22114, 24047,  3708,  1137,
         14213,  6124,  4643, 37465, 33895, 31423, 19064, 37465,  5341, 26068,
         22140, 10910, 10299, 31423,  5383, 14526, 31423, 27302, 11531, 38938,
         33720,  1553, 22696, 12822, 17547, 18414, 2

In [49]:
class GenerizeText(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_size, residual = 'RNN', pretrained_embeddings = []):
    super().__init__()

    if len(pretrained_embeddings) > 0:
      self.embeddings = nn.Embedding.from_pretrained(pretrained_embeddings)
    else:
      self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
    
    self.residual = residual.lower()

    if self.residual == 'rnn':
      self.rl = nn.RNN(input_size = embedding_dim, 
                        hidden_size = hidden_size,
                        num_layers = 2, 
                        nonlinearity = 'relu',
                        batch_first = True,
                        dropout = 0.25)
      
    elif self.residual == 'lstm':
      self.rl = nn.LSTM(input_size = embedding_dim, 
                        hidden_size = hidden_size,
                        num_layers = 2,
                        batch_first = True)  
                        # dropout = 0.5)    

    elif self.residual == 'gru':
      self.rl = nn.GRU(input_size = embedding_dim, 
                        hidden_size = hidden_size,
                        num_layers = 2, 
                        batch_first = True) 
                        # dropout = 0.5)

    # self.fc = nn.Linear(hidden_size, n_classes)
    self.fc = nn.Sequential(nn.Linear(hidden_size, vocab_size))
    
  def forward(self, input):
    embed = self.embeddings(input)
    # print('Embed: ', embed.shape)
    if self.residual == 'lstm':
      out, (hn, cn) = self.rl(embed)
      # out = hn[0]
    else:
      out, hidden = self.rl(embed)
      # out = hidden[0]
    # print('Output RNN: ', out.shape)
    out = torch.reshape(out, (-1, out.shape[-1]))
    # print('Output reshape: ', out.size())
    out = self.fc(out)
    return out

In [50]:
def generateText(model, start_seq: list, vocab: Vocab, device: str):
  indices = [2]
  for word in start_seq:
    indices.append(vocab.token_to_idx[word])

  h_t = None
  sample_size = vocab.max_seq_len

  for step in range(sample_size):
    x_t = torch.LongTensor([indices[step]]).unsqueeze(dim=0).to(device=device)
    x_emp = model.embeddings(x_t)
    out_t, h_t = model.rl(x_emp, h_t)
    out_t = out_t.squeeze(dim=1)
    predictions = model.fc(out_t).softmax(dim=1)
    char_idx = torch.multinomial(predictions, num_samples=1)
    if step >= 2:
      indices.append(char_idx.item())
  
  text = ''
  for idx in range(0, len(indices), 15):
    if idx + 15 < len(indices):
      text += ' '.join(vocab.idx_to_token[token_idx] for token_idx in indices[idx: idx + 15] if token_idx != 2) + '\n'
    else:
      text += ' '.join(vocab.idx_to_token[token_idx] for token_idx in indices[idx:] if token_idx != 2) + '\n'
  return text #' '.join(vocab.idx_to_token[token_idx] for token_idx in indices if token_idx != 2)

In [51]:
dataset = NameRusDataset(list(df['Part_text']), vocab)

dataloader = DataLoader(dataset, batch_size=len(dataset), shuffle=True, drop_last=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
model = GenerizeText(vocab_size=vocab.vocab_len,
                     hidden_size=128,
                     embedding_dim=100,
                     residual='rnn').to(device)

criteriation = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=5*10**(-4))

loss_history = list()
epochs_history = list()

n_epochs = 20 + 1

for epoch in range(1, n_epochs):
  epoch_loss = 0
  model.train()
  for X_batch, y_batch in dataloader:
    X_batch, y_batch = X_batch.to(device=device), y_batch.to(device=device) # y_barch: batch_size x seq_len
    predictions = model(X_batch)
    # predictions: batch_size*seq_len x vocab_size
    y_batch = y_batch.reshape(-1)
    # y_batch: batch_size*seq_len
    loss = criteriation(predictions, y_batch)
    loss.backward()
    epoch_loss += loss.item()

    optimizer.step()
    clip_grad_norm_(model.parameters(), 0.5)
    optimizer.zero_grad()

  loss_history.append(epoch_loss/len(dataloader))
  epochs_history.append(epoch)

  model.eval()
  print(f"Epoch {epochs_history[-1]}/{n_epochs-1} loss = {loss_history[-1]} Example:\n {generateText(model, start_seq=['Ай', 'Рядовой', 'Моргенштерн'], vocab=vocab, device=device)}")

RuntimeError: ignored